# Property Business History EDA

## Checking to see if all listing_keys are in CRMLS and PBH

In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

In [5]:
euphrates = pd.read_csv('data/crmls_euphrates_listingkey_status.csv')
for i in range(2,5):
    euphrates = pd.concat([euphrates, pd.read_csv(f'data/crmls_euphrates_listingkey_status_{i}.csv')])

len(euphrates)

12807418

In [6]:
euphrates.head()

,source_id,listing_key,standard_status
0,P835842:CRM,616289,S
1,IV16108764:CRM,13380498,S
2,ML81015975:MLSL,12497076,S
3,V0-219013515:CRF,363699119,K
4,SR22091434:CRM,368857850,K


In [7]:
pbh_last_status = pd.read_csv('data/property_business_history_last_status.csv')
pbh_last_status.head()

,businesshistorykeynumeric,deletedyn,effectivetimestamp,inputentryorder,listingkeynumeric,modificationtimestamp,originatingsystemid,price,standardstatus
0,1412532,False,2014-11-25 16:00:00,0.0,8584872.0,2024-10-31 07:34:14,CDAR,475000.0,S
1,1412563,False,2015-02-25 16:00:00,0.0,8585038.0,2024-10-31 07:34:14,CDAR,300000.0,S
2,1212547,False,2024-10-07 17:00:00,6.0,373180988.0,2024-10-08 15:04:03,CRF,365000.0,S
3,1212571,False,2024-06-25 17:00:00,2.0,372478834.0,2024-10-08 15:04:03,CRM,660000.0,S
4,1225315,False,2024-10-08 17:00:00,2.0,373092667.0,2024-10-09 13:06:37,CDAR,5000.0,S


In [10]:
pbh_last_status[~pbh_last_status['listingkeynumeric'].isin(euphrates['listing_key'])].to_clipboard(index=False)

In [9]:
len(pbh_last_status)

663954

In [11]:
merged = euphrates.merge(pbh_last_status, how='inner', left_on='listing_key', right_on='listingkeynumeric')

In [12]:
len(merged)

663606

In [13]:
merged.head()

,source_id,listing_key,standard_status,businesshistorykeynumeric,deletedyn,effectivetimestamp,inputentryorder,listingkeynumeric,modificationtimestamp,originatingsystemid,price,standardstatus
0,P0-214016727:CRF,6438963,S,200304,False,2014-06-11 17:00:00,0.0,6438963.0,2024-04-16 15:51:12,CRF,716000.0,S
1,36025258:SAND,366061570,S,1127506,False,2003-06-29 17:00:00,0.0,366061570.0,2024-09-30 05:14:05,SAND,443000.0,S
2,19492172PS:CDAR,363070931,X,1220185,False,2020-07-24 17:00:00,0.0,363070931.0,2024-10-09 06:28:12,CDAR,635000.0,X
3,213007737DA:CDAR,8667952,S,1396621,False,2014-01-09 16:00:00,0.0,8667952.0,2024-10-30 05:25:12,CDAR,375000.0,S
4,219011095DA:CDAR,362468966,X,1355777,False,2019-05-23 17:00:00,0.0,362468966.0,2024-10-23 07:38:14,CDAR,7300.0,X


In [16]:
map_status = {
    'W': 'Withdrawn',
    'X': 'Expired',
    'A': 'Active',
    'S': 'Closed',
    'K': 'Canceled',
    'H': 'Hold',
    'C': 'Coming Soon',
    'P': 'Pending'
}
merged['map_status'] = merged['standard_status'].map(map_status)

In [17]:
merged[merged['standardstatus']!=merged['standard_status']]

,source_id,listing_key,standard_status,businesshistorykeynumeric,deletedyn,effectivetimestamp,inputentryorder,listingkeynumeric,modificationtimestamp,originatingsystemid,price,standardstatus,map_status
39,ML81950032:MLSL,372245163,K,800251,False,2024-08-14 05:45:41,4.0,372245163.0,2024-08-14 13:13:45,MLSL,1555000.0,A,Canceled
41,23340999:CL,372245161,X,113779,False,2024-03-14 12:56:36,1.0,372245161.0,2024-03-14 19:58:49,CL,799000.0,A,Expired
2016,23288512:CL,371668421,K,38533,False,2023-07-09 17:00:00,0.0,371668421.0,2024-02-16 15:20:04,CL,1099000.0,A,Canceled
2091,SW23194760:CRM,372055790,X,406424,False,2024-06-14 00:01:47,1.0,372055790.0,2024-06-14 05:18:59,CRM,499000.0,A,Expired
2093,41046252:CCBE,372245218,S,25087,False,2024-02-12 16:00:00,1.0,372245218.0,2024-02-13 11:01:59,CCBE,1882845.0,P,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
661774,V1-20016:CRF,371925129,K,45102,False,2024-01-25 16:00:00,0.0,371925129.0,2024-02-20 11:24:00,CRF,235000.0,S,Canceled
662024,219074360DA:CDAR,368209329,K,739007,False,2024-08-07 06:45:51,1.0,368209329.0,2024-08-07 07:42:09,CDAR,15000.0,A,Canceled
662159,23313393:CL,371945678,K,143328,False,2024-03-27 01:11:26,4.0,371945678.0,2024-03-27 08:16:33,CL,5904.0,A,Canceled
662267,23307839:CL,371887524,X,66618,False,2024-02-22 16:00:00,1.0,371887524.0,2024-02-27 13:57:16,CL,4950000.0,U,Expired


In [20]:
merged.groupby(['standard_status', 'standardstatus'])['listingkeynumeric'].count().to_clipboard()

In [ ]:
merged[merged['standardstatus']!=merged['standard_status']].groupby(['standard_status', 'standardstatus'])['listingkeynumeric'].count().sum()

np.int64(64212)

In [22]:
len(merged)

663606

In [23]:
pbh_last_status['modificationtimestamp'].max()

'2024-11-01 09:29:20'

In [24]:
merged[merged['standardstatus']!=merged['standard_status']]['modificationtimestamp'].max()

'2024-11-01 09:29:20'

In [30]:
euphrates['listing_id'] = [x.split(':')[0] for x in euphrates['source_id']]
len(pbh_last_status[pbh_last_status['listingkeynumeric'].isin(euphrates['listing_id'])])

0

In [29]:
len(pbh_last_status[pbh_last_status['listingkeynumeric'].isin(euphrates['listing_key'])])

663606

## Checking Avg Date

In [2]:
euphrates = pd.read_csv(f'data/crmls_listings_status_dates.csv', low_memory=False)

In [3]:
for i in tqdm(range(2,13)):
    euphrates = pd.concat([euphrates, pd.read_csv(f'data/crmls_listings_status_dates_{i}.csv', low_memory=False)])
len(euphrates)

  0%|          | 0/11 [00:00<?, ?it/s]

12810208

In [4]:
euphrates.head()

,source_id,listing_key,standard_status,listing_date,expiration_date,contract_date,hold_date,close_date,on_market_date,off_market_date,withdraw_date
0,ML81932538:MLSL,371605375,S,2023-06-21,2023-09-21,2023-06-30,NaN,2023-07-19,2023-06-21,2023-07-01,NaN
1,23284015:CL,371613946,S,2023-06-23,2023-11-05,2023-07-17,NaN,2023-07-26,2023-06-23,2023-07-17,NaN
2,41031148:CCBE,371613940,S,2023-06-23,NaN,2023-07-02,NaN,2023-07-28,NaN,2023-07-02,NaN
3,41031149:CCBE,371613941,K,2023-06-23,NaN,NaN,NaN,NaN,NaN,2023-07-21,NaN
4,NDP2304824:CRP,371613947,X,2023-06-23,2023-10-01,NaN,NaN,NaN,2023-06-23,2023-10-02,NaN


In [5]:
date_cols = [x for x in euphrates.columns if 'date' in x]
for i in tqdm(date_cols):
    euphrates[i] = pd.to_datetime(euphrates[i], format='%Y-%m-%d', errors='coerce').values.astype(np.int64)

euphrates.head()

  0%|          | 0/8 [00:00<?, ?it/s]

,source_id,listing_key,standard_status,listing_date,expiration_date,contract_date,hold_date,close_date,on_market_date,off_market_date,withdraw_date
0,ML81932538:MLSL,371605375,S,1687305600000000000,1695254400000000000,1688083200000000000,-9223372036854775808,1689724800000000000,1687305600000000000,1688169600000000000,-9223372036854775808
1,23284015:CL,371613946,S,1687478400000000000,1699142400000000000,1689552000000000000,-9223372036854775808,1690329600000000000,1687478400000000000,1689552000000000000,-9223372036854775808
2,41031148:CCBE,371613940,S,1687478400000000000,-9223372036854775808,1688256000000000000,-9223372036854775808,1690502400000000000,-9223372036854775808,1688256000000000000,-9223372036854775808
3,41031149:CCBE,371613941,K,1687478400000000000,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,1689897600000000000,-9223372036854775808
4,NDP2304824:CRP,371613947,X,1687478400000000000,1696118400000000000,-9223372036854775808,-9223372036854775808,-9223372036854775808,1687478400000000000,1696204800000000000,-9223372036854775808


In [6]:
euphrates_transposed = euphrates.drop(columns=['source_id', 'standard_status']).transpose()
euphrates_transposed.columns = list(euphrates_transposed.loc['listing_key'])

In [7]:
euphrates_transposed.drop('listing_key', axis=0, inplace=True)
euphrates_transposed.head()

,371605375,371613946,371613940,371613941,371613947,371613944,4116562,4116428,4116594,4116604,...,371613739,4116432,4116561,4116571,4116209,4116547,4116524,4116574,4116552,4116299
listing_date,1687305600000000000,1687478400000000000,1687478400000000000,1687478400000000000,1687478400000000000,1687478400000000000,1176681600000000000,1237939200000000000,1176681600000000000,1118793600000000000,...,1687478400000000000,1237248000000000000,1160956800000000000,1176163200000000000,1236988800000000000,1181692800000000000,1237852800000000000,1222819200000000000,1176681600000000000,1237507200000000000
expiration_date,1695254400000000000,1699142400000000000,-9223372036854775808,-9223372036854775808,1696118400000000000,1716076800000000000,1199059200000000000,1238198400000000000,1199059200000000000,1194480000000000000,...,1719187200000000000,1262217600000000000,1192492800000000000,1191974400000000000,1252886400000000000,1199059200000000000,1253750400000000000,1230681600000000000,1191542400000000000,1245456000000000000
contract_date,1688083200000000000,1689552000000000000,1688256000000000000,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,1237852800000000000,-9223372036854775808,-9223372036854775808,...,1693008000000000000,1251763200000000000,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,1187827200000000000,-9223372036854775808
hold_date,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,...,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808
close_date,1689724800000000000,1690329600000000000,1690502400000000000,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,1237939200000000000,-9223372036854775808,-9223372036854775808,...,1695945600000000000,1271376000000000000,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,-9223372036854775808,1190505600000000000,1241136000000000000


In [8]:
euphrates_transposed = euphrates_transposed.replace(-9223372036854775808, np.nan)

In [9]:
avg_euphrates_date = euphrates_transposed.mean(axis=0)

In [10]:
avg_euphrates_date = pd.to_datetime(avg_euphrates_date, errors='coerce')
avg_euphrates_date.head()

371605375   2023-07-14 04:00:00
371613946   2023-07-29 00:00:00
371613940   2023-07-06 06:00:00
371613941   2023-07-07 00:00:00
371613947   2023-08-12 06:00:00
dtype: datetime64[ns]

In [13]:
pbh_timestamps = pd.read_csv('data/pbh_timestamps.csv')
pbh_timestamps.head()

,listingkeynumeric,effectivetimestamp
0,373539669.0,2024-10-30 16:56:41
1,373124395.0,2024-10-30 16:55:54
2,373022371.0,2024-09-10 10:05:26
3,373252297.0,2024-09-09 00:00:00
4,372481570.0,2024-09-10 10:14:16


In [14]:
pbh_timestamps['effectivetimestamp'] = pd.to_datetime(pbh_timestamps['effectivetimestamp'], errors='coerce').astype(np.int64)
pbh_timestamps['effectivetimestamp'] = pbh_timestamps['effectivetimestamp'].replace(-9223372036854775808, np.nan)

In [15]:
pbh_avg_date = pd.to_datetime(pbh_timestamps.groupby('listingkeynumeric')['effectivetimestamp'].mean(), errors='coerce')
pbh_avg_date.name = 'pbh_avg_date'
avg_euphrates_date.name = 'euphrates_avg_date'

In [17]:
avg_euphrates_date

371605375   2023-07-14 04:00:00
371613946   2023-07-29 00:00:00
371613940   2023-07-06 06:00:00
371613941   2023-07-07 00:00:00
371613947   2023-08-12 06:00:00
                    ...        
4116547     2007-10-25 00:00:00
4116524     2009-07-24 16:00:00
4116574     2008-11-30 16:00:00
4116552     2007-12-03 04:48:00
4116299     2009-05-03 00:00:00
Name: euphrates_avg_date, Length: 12810208, dtype: datetime64[ns]

In [20]:
compare_dates = pd.concat([pbh_avg_date, avg_euphrates_date], axis=1).dropna()
compare_dates.head()

,pbh_avg_date,euphrates_avg_date
574359.0,2003-09-10 17:00:00,2003-08-01 16:00:00
575398.0,2012-08-28 17:00:00,2012-09-10 19:12:00
576194.0,2002-05-16 17:00:00,2002-06-08 08:00:00
579107.0,2012-10-28 17:00:00,2012-11-18 19:12:00
579335.0,2012-12-27 16:00:00,2012-11-09 19:12:00


In [24]:
compare_dates['days_diff']= compare_dates['euphrates_avg_date'] - compare_dates['pbh_avg_date']

In [ ]:
compare_dates['days_diff'].mean()

Timedelta('-13 days +08:48:20.487353337')

In [25]:
compare_dates.head()

,pbh_avg_date,euphrates_avg_date,days_diff
574359.0,2003-09-10 17:00:00,2003-08-01 16:00:00,-41 days +23:00:00
575398.0,2012-08-28 17:00:00,2012-09-10 19:12:00,13 days 02:12:00
576194.0,2002-05-16 17:00:00,2002-06-08 08:00:00,22 days 15:00:00
579107.0,2012-10-28 17:00:00,2012-11-18 19:12:00,21 days 02:12:00
579335.0,2012-12-27 16:00:00,2012-11-09 19:12:00,-48 days +03:12:00


In [26]:
compare_dates['days_diff'] = [abs(x) for x in compare_dates['days_diff']]
compare_dates.head()

,pbh_avg_date,euphrates_avg_date,days_diff
574359.0,2003-09-10 17:00:00,2003-08-01 16:00:00,40 days 01:00:00
575398.0,2012-08-28 17:00:00,2012-09-10 19:12:00,13 days 02:12:00
576194.0,2002-05-16 17:00:00,2002-06-08 08:00:00,22 days 15:00:00
579107.0,2012-10-28 17:00:00,2012-11-18 19:12:00,21 days 02:12:00
579335.0,2012-12-27 16:00:00,2012-11-09 19:12:00,47 days 20:48:00


In [27]:
from datetime import timedelta

In [31]:
def days_hist(x):
    if x <= timedelta(days=1):
        return '0) 1 day'
    elif x <= timedelta(days=7):
        return '1) 1 week'
    elif x <= timedelta(days=14):
        return '2) 2 weeks' 
    elif x <= timedelta(days=30):
        return '3) 1 month'
    elif x <= timedelta(weeks=26):
        return '4) 6 months'
    elif x <= timedelta(days=365):
        return '5) 1 year' 
    else:
        return '6) > 1 year'
compare_dates['days_hist'] = [days_hist(x) for x in compare_dates['days_diff']]

In [35]:
compare_dates.groupby('days_hist').count().to_clipboard()